In [1]:
import pandas as pd
import numpy as np
import datetime
import os
import glob
os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2018_Q4/Circ_Test_Con'

In [2]:
test_week_list_2018=[datetime.date(2018,9,15)+datetime.timedelta(days=x*7) for x in range(8)]
test_week_list_2017=[x+datetime.timedelta(days=-52*7) for x in test_week_list_2018]

In [3]:
historical_rewards=pd.read_csv("/home/jian/Projects/Big_Lots/Loyal_members/loyalty_sales_data/From_Sp/combinedtransactions_0811.csv",
                               dtype=str,usecols=['customer_id_hashed','transaction_date','transaction_id','location_id','total_transaction_amt','transaction_time'])
historical_rewards=historical_rewards.drop_duplicates()
historical_rewards['transaction_date']=historical_rewards['transaction_date'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())

In [4]:
historical_rewards_2017=historical_rewards[(historical_rewards['transaction_date']>=(min(test_week_list_2017)-datetime.timedelta(days=6))) &\
                                      (historical_rewards['transaction_date']<=max(test_week_list_2017))]
del historical_rewards

In [5]:
biweek_rewards_2018_1=pd.read_table("/home/jian/Projects/Big_Lots/Loyal_members/loyalty_sales_data/MediaStormSalesBiWeekly20180925-132958-543.txt",dtype=str,sep="|")
biweek_rewards_2018_2=pd.read_table("/home/jian/Projects/Big_Lots/Loyal_members/loyalty_sales_data/MediaStormSalesBiWeekly20181009-132856-945.txt",dtype=str,sep="|")
biweek_rewards_2018_3=pd.read_table("/home/jian/Projects/Big_Lots/Loyal_members/loyalty_sales_data/MediaStormSalesBiWeekly20181023-132316-829.txt",dtype=str,sep="|")
biweek_rewards_2018_4=pd.read_table("/home/jian/Projects/Big_Lots/Loyal_members/loyalty_sales_data/MediaStormSalesBiWeekly20181106-132358-296.txt",dtype=str,sep="|")

In [6]:
biweek_rewards_2018=biweek_rewards_2018_1.append(biweek_rewards_2018_2).append(biweek_rewards_2018_3).append(biweek_rewards_2018_4)
del biweek_rewards_2018_1
del biweek_rewards_2018_2
del biweek_rewards_2018_3
del biweek_rewards_2018_4
del biweek_rewards_2018['merch_cat']
del biweek_rewards_2018['total_transaction_units']
biweek_rewards_2018=biweek_rewards_2018.drop_duplicates()
biweek_rewards_2018['transaction_dt']=biweek_rewards_2018['transaction_dt'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())



In [8]:
historical_rewards_2017['total_transaction_amt']=historical_rewards_2017['total_transaction_amt'].astype(float)
biweek_rewards_2018['total_transaction_amt']=biweek_rewards_2018['total_transaction_amt'].astype(float)
historical_rewards_2017=historical_rewards_2017.rename(columns={"transaction_date":"transaction_dt"})
rewards_by_day_2017=historical_rewards_2017.groupby(['location_id','transaction_dt'])['total_transaction_amt'].sum().to_frame().reset_index()
rewards_by_day_2018=biweek_rewards_2018.groupby(['location_id','transaction_dt'])['total_transaction_amt'].sum().to_frame().reset_index()



In [9]:
def week_end_dt(x):
    if x.weekday()==6:
        y=x+datetime.timedelta(days=6)
    else:
        y=x+datetime.timedelta(days=(5-x.weekday()))
    return y
rewards_by_day_2017['week_end_dt']=rewards_by_day_2017['transaction_dt'].apply(lambda x: week_end_dt(x))
rewards_by_day_2018['week_end_dt']=rewards_by_day_2018['transaction_dt'].apply(lambda x: week_end_dt(x))

In [10]:
rewards_by_week_2017=rewards_by_day_2017.groupby(['location_id','week_end_dt'])['total_transaction_amt'].sum().to_frame().reset_index()
rewards_by_week_2018=rewards_by_day_2018.groupby(['location_id','week_end_dt'])['total_transaction_amt'].sum().to_frame().reset_index()


In [11]:
rewards_by_week_2017_wide=rewards_by_week_2017.pivot_table(index="location_id",columns="week_end_dt",values="total_transaction_amt").reset_index()
rewards_by_week_2018_wide=rewards_by_week_2018.pivot_table(index="location_id",columns="week_end_dt",values="total_transaction_amt").reset_index()



In [12]:
writer=pd.ExcelWriter("/home/jian/Projects/Big_Lots/Analysis/2018_Q4/Circ_Test_Con/BL_rewards_by_store_week_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")
rewards_by_week_2017_wide.to_excel(writer,"rewards_2017",index=False)
rewards_by_week_2018_wide.to_excel(writer,"rewards_2018",index=False)
writer.save()

In [ ]:
# ,usecols=['customer_id_hashed','transaction_date','transaction_id','location_id','total_transaction_amt','transaction_time]

In [5]:
test_week_list_2018=[datetime.date(2018,9,15)+datetime.timedelta(days=x*7) for x in range(8)]
test_week_list_2017=[x+datetime.timedelta(days=-52*7) for x in test_week_list_2018]
start_2017=min(test_week_list_2017)-datetime.timedelta(days=6)
date_2017_list_str=[str(start_2017+datetime.timedelta(days=x)) for x in range(56)]

In [11]:
header_df=pd.read_table("/home/jian/Projects/Big_Lots/Loyal_members/loyalty_sales_data/MediaStormSalesBiWeekly20180925-132958-543.txt",dtype=str,sep="|",nrows=100)

very_early_rewards=pd.read_table("/home/jian/Projects/Big_Lots/Loyal_members/loyalty_sales_data/MediaStormCustDtl.txt",
                                 dtype=str,sep=",",header=None)
very_early_rewards.columns=header_df.columns
del very_early_rewards['merch_cat']
very_early_rewards=very_early_rewards.drop_duplicates()

In [13]:
very_early_rewards=very_early_rewards[very_early_rewards['transaction_dt'].isin(date_2017_list_str)]
very_early_rewards=very_early_rewards.reset_index()
del very_early_rewards['index']
very_early_rewards['total_transaction_amt']=very_early_rewards['total_transaction_amt'].astype(float)

In [14]:
very_early_rewards.head(3)

,customer_id_hashed,transaction_dt,transaction_time,transaction_id,location_id,total_transaction_amt,total_transaction_units
0,31543729,2017-09-10,17:20,6571,5324,26.00,3
1,22163857,2017-09-10,14:12,1945,5096,17.65,26
2,25635953,2017-09-10,20:06,2877,5298,17.45,5


In [15]:
very_early_rewards_1701=very_early_rewards[very_early_rewards['location_id']=="1701"]


In [16]:
very_early_rewards_1701['total_transaction_amt'].sum()

78769.699999999997

In [17]:
len(very_early_rewards_1701['transaction_dt'].unique())

56